# Start Up: 
Before Running the cell below, you must ensure that these have been run in Terminal **IN ORDER** : 
- conda update -n base -c defaults conda 

    - cd SageMaker
    
      - cd yelp-dataset-challenge-1-ds
      
         - conda env create -f environment.yml
          
            - source activate ydc1 
                
                - pip install python-decouple
                  
                  - pip install pprintpp
                  
# Spacy Installs: 

   - python -m spacy download en_core_web_lg

        - python -m spacy link en_core_web_lg en

In [80]:
import json

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
#import s3
#from pprintpp import pprint as pp
from sklearn.externals import joblib
nlp = spacy.load('en')

# Load in Bucket
#bucket = s3.Bucket('yelpchallenge1')
# Look inside the bucket.
#bucket.contents

In [ ]:
                    ### ***** DO NOT RUN. ******* #### 
                  ### ***** ALREADY INSTALLED. ****** ###

# Only have to run this once.
# Installs the .csv 'Locally' on SageMaker Instance

#bucket.get('datasets/df.csv', 'df.csv')

# Installing user.json 'Locally'
#bucket.get('datasets/user.json', 'user.json')


    # Load in Bucket
# bucket = s3.Bucket('yelpchallenge1')
    # Look inside the bucket.
# bucket.contents

# Getting Started: Imports

In [4]:
    # Read-in final.csv
final_df = pd.read_csv('final_df.csv')

    # Read-in df.csv
#df = pd.read_csv('df.csv', nrows=250000)


    # Read-in review_df.csv
#review_df = pd.read_csv('review.csv')

    # Read-in dtm_final.csv (FINAL)
dtm_final = pd.read_csv('dtm_final.csv')

    # import Vectorizer Model
#vect2 = joblib.load('vect_2.sav')

In [5]:
dtm_final.head(5)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,-PRON-,-PRON-,-PRON- -PRON-,-PRON- a,-PRON- all,-PRON- also,-PRON- and,...,worth,would,would be,would have,would not,wrong,year,yelp,yet,stars
0,0,0,0,0.069829,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,1.0
1,1,1,1,0.343331,0.0,0.000000,0.0,0.0,0.0,0.047998,...,0.052035,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,5.0
2,2,2,2,0.314034,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.093163,0.0,0.000000,5.0
3,3,3,3,0.080641,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.087372,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,5.0
4,4,4,4,0.515313,0.0,0.020392,0.0,0.0,0.0,0.043038,...,0.000000,0.029004,0.0,0.0,0.02601,0.027762,0.021839,0.0,0.028812,1.0


In [6]:
dtm_final = dtm_final.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [7]:
dtm_final = dtm_final.drop(columns = ['stars'])

In [8]:
dtm_final.head(1)

,-PRON-,-PRON-,-PRON- -PRON-,-PRON- a,-PRON- all,-PRON- also,-PRON- and,-PRON- be,-PRON- but,-PRON- can,...,work,worth,would,would be,would have,would not,wrong,year,yelp,yet
0,0.069829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
dtm_final.shape

(95000, 773)

In [77]:
final_df.columns

Index(['useful', 'funny', 'cool', 'text', 'tokens'], dtype='object')

In [80]:
# Dropping All Rows with 0 in Interactions Column 
final_df = final_df[final_df.useful !=0]
final_df = final_df[final_df.funny !=0]
final_df = final_df[final_df.cool !=0]

In [81]:
final_df.head(5)

,useful,funny,cool,text,tokens
6,5.0,4.0,5,Tracy dessert had a big name in Hong Kong and ...,"['tracy', 'dessert', 'hong', 'kong', 'markham'..."
7,3.0,1.0,1,This place has gone down hill. Clearly they h...,"['place', 'gone', 'hill', 'clearly', 'staff', ..."
17,1.0,7.0,1,I love chinese food and I love mexican food. W...,"['love', 'chinese', 'food', 'love', 'mexican',..."
21,9.0,6.0,9,If you are looking for the best pierogies in P...,"['looking', 'best', 'pierogies', 'pittsburgh',..."
26,7.0,1.0,1,"Unfortunately, I must recommend not to conduct...","['unfortunately', 'recommend', 'conduct', 'bus..."


In [82]:
final_df.shape

(11904, 5)

In [83]:
final_df.to_csv(index=True)
final_df.to_csv(r'final_df.csv')

# Model Prep: 

**Description**: 

Combining based on their *Unique Account ID's.*
The end Product will be One DataFrame Consisting of Each Account:
- **Name**, 
- **User_ID**,
- **Review_ID**,
- **Text**,
- **That Users respective review(s)**,
- **Interactions that Review (i.e: Cool, Funny, Useful)**  

The goal of the model is to have the ability to type in the Review you are wanting to post on Yelp, and give the User the ability to Predict What type of Interaction they would potentially receive and Total Number of each interaction. The model Accuracy will be Displayed beside the Prediction. 

In [10]:
# Imports: 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [84]:
# Creating Variables for Training 
interactions1 = final_df['useful']

In [90]:
# Splitting Dataset into training and test set

    # Using 10,000 Rows. 

reviews = final_df['text'].iloc[0:10000]
interactions1 = interactions1.iloc[0:10000]

In [86]:
# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [87]:
# Defining
# tokenizer
def tokenize(document):
    doc = nlp(document)
    return [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

In [88]:
# Creating Fake Review for Testing: 
fake_review = """
After finshing our dinner we heard a loud crash come from the back of the place The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation given that I was completley drunk and could not stop laughing. The beer was a little flat, the 'chicken' was a little gamey. 
Service was great 4 Stars.
"""

In [89]:
# tokenizer
vect_review = tfidf.fit_transform(tokenize(fake_review))

In [93]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()


# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])


#Tuning
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, 0.05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs= -1, verbose=1)
grid_search.fit(reviews, interaction)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/Users/haden/anaconda3/envs/ydc1/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  6.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [33]:
grid_search.best_score_

0.5387

In [34]:
grid_search.best_params_

{'clf__max_depth': 15,
 'clf__n_estimators': 10,
 'vect__max_df': 1.0,
 'vect__max_features': 500,
 'vect__min_df': 0.02}

In [44]:
# Fake Review Interaction 'Useful' Prediction: 

useful_pred = grid_search.predict([r'vect_review'])
print(useful_pred)

[0.]


In [45]:
#Pickling Model with JobLib
joblib.dump(useful_pred, 'useful_clf.joblib')

['useful_clf.joblib']

In [65]:
# Creating Fake Review for Testing: 
fake_review2 = """Big props to Joseph at the drive-thru window for really representing the Starbucks brand well and pushing for app downloads.
He makes for a great brand ambassador! Expect long lines in the drive-through but friendly service. I was surprised and curious to see the sous vide egg and bacon option now on the menu. Located along capital Avenue on your way to downtown the spot seems to be quite popular for those grabbing a coffee before heading into the office.
"""

In [72]:
# tokenizer
vect_review3 = tfidf.fit_transform(tokenize(fake_review))

In [73]:
useful_pred2 = grid_search.predict([r'vect_review2'])
print(useful_pred2)

[0.]


In [63]:
# tokenizer
vect_review2 = tfidf.fit_transform(tokenize(final_df['text'].iloc[7]))

In [64]:
useful_pred2 = grid_search.predict([r'vect_review2'])
print(useful_pred2)

[0.]


In [53]:
final_df['text'].iloc[7]

"Here's why I don't write reviews for Chinese restaurants: I have to look up their English names... It's so popular that I didn't try it until recently. The bubble tea here is really good. It's pricier than the one in Rose tea, but you also get more. I like that noodles with pork sauce. I've tried both the thin and wide/thick noodles. The wide one is less chewy and are kind of sticky. I prefer the thin one for this dish.Dan dan noodles are different from what I was expecting. My friend liked it but it's just okay for me.The noodle with wonton is flavorless to me. The soup dumplings are good but not amazing. If you've never had it before, give it a try and you probably will like it.I'd like to try noodles with beef soup next time.The appetizers are too pricy in my opinion. I won't pay $6 for a small plate of vegetables again...The owner is a nice guy and I enjoyed chatting with him a little bit during dinners."

In [62]:
final_df.iloc[4]

useful                                                    7
funny                                                     0
cool                                                      0
text      Today was my second out of three sessions I ha...
tokens    ['today', 'second', 'sessions', 'paid', 'sessi...
Name: 4, dtype: object

# NEW Model Prep: As of 1/08/2019

In [11]:
# Dropping Unwanted Columns
#final_df = final.drop(columns=['Unnamed: 0', 'user_id', 'review_id', 'date'])
# Final_df.csv File was too Large. Cutting Down to First 95,000 Rows.
final_df = final_df.iloc[0:95000]

# Save Cleaned and Trimmed final_df.csv 
final_df.to_csv(index=True)
final_df.to_csv(r'final_df.csv')

# final_df.head(5)

In [13]:
final_df = final_df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [14]:
dtm = dtm_final

In [15]:
dtm.shape

(95000, 773)

In [12]:
dtm_final = dtm.iloc[0:95000]
# Save Trimmed dtm_final.csv 
dtm_final.to_csv(index=True)
dtm_final.to_csv(r'dtm_final.csv')

In [20]:
dtm_final.shape

(95000, 773)

# Testing Random Forest Regression Model: 

In [16]:
# Imports:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np 
import matplotlib.pyplot as plt
# import Vectorizer Model
#vect = joblib.load('vect_2.sav')

In [17]:
interactions = final_df[['useful', 'funny', 'cool']]

In [23]:
# Splitting Dataset into training and test set
X = dtm_final
y = interactions

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 0)

In [ ]:
# Fitting Random Forest Regression to Dataset
regressor = RandomForestRegressor(n_estimators = 15, random_state = 0)
regressor.fit(X_train, y_train)

In [27]:
#Pickling Model with JobLib
from sklearn.externals import joblib
joblib.dump(regressor, 'regressor.joblib')

['regressor.joblib']

In [18]:
# Import Model 
regressor = joblib.load('regressor.joblib')

In [19]:
# Creating Fake Review for Testing: 
fake_review = """
After finshing our dinner, we heard a loud crash come from the back of the place. The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation, given that I was completley drunk and could not stop laughing. The beer was a little flat, the chicken was a little gamey. 
Service was great though. four Stars.
"""

In [ ]:
interaction_pred = regressor.predict(fake_review)
print(fake_review)

# Testing Different Regressor: 

In [28]:
# Imports:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np 
import matplotlib.pyplot as plt

In [33]:
# Only Using ONE Interaction: 
interactions2 = final_df['useful']

# Splitting Dataset into training and test set
X = dtm_final
y = interactions2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 0)

In [ ]:
# Fitting Random Forest Regression to Dataset
regressor2 = RandomForestRegressor(n_estimators = 5, random_state = 0)
regressor2.fit(X_train, y_train)

In [ ]:
# Creating Fake Review for Testing: 
fake_review = ["""
After finshing our dinner, we heard a loud crash come from the back of the place. The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation, given that I was completley drunk and could not stop laughing. The beer was a little flat, the chicken was a little gamey. 
Service was great though. 4 Stars.
"""]

In [ ]:
# tokenizer
vect_review = tfidf.fit_transform(tokenize(fake_review))